<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>    

# 7.5_ T5 para generación de resúmenes

La generación de resúmenes es una las aplicaciones de PLN más importantes. Es una tarea muy difícil que plantea varios desafíos, como identificar el contenido importante y sintetizarlo todo en un nuevo texto.

En este ejercicio, ajustaremos el transformer T5 a la tarea de generación de resúmenes. Este modelo tiene una arquitectura codificador-decodificador. Aunque puede ser utilizado para cualquier rtarea de NLP, es una buena opción en tareas de generación de textos, como la generaciónd e resúmenes.

El dataset que vamos a utilizar es XSum (https://huggingface.co/datasets/xsum) un dataset formado por 226.711 noticias de la BBC y sus correspondientes resúmenes (cada resumen es una oración). Los artículos cubren una amplia variedad de dominios (por ejemplo, noticias, política, deportes, clima, negocios, tecnología, ciencia, salud, familia, educación, entretenimiento y artes).



## Instalar librerías

Además de instalar las librerías de transformers y datastes, también instalaremos la librería keras_nlp, que será utilizada para calcular la métrica rouge_L para la tarea de generación de resúmenes:

In [1]:
!pip install -q transformers[torch] datasets rouge-score keras_nlp

## Cargar el dataset

Vamos a cargar el dataset XSum desde Hugging Face. Puede tardar unos minutos....


In [2]:
from datasets import load_dataset
dataset = load_dataset("xsum")
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

Vemos que el split train contiene aproximadamente un 90% del dataset original (204.045 noticias), mientras que los conjuntos de validación y test tienen únicamente un 5% cada uno de ellos.

Aunque disponer de datasets de gran tamaño siempre es una ventaja, para este ejercicio, únicamente usaremos una pequeña porción del dataset, para que el modelo pueda ser entrenado en Google Colab (sin usar el servicio Pro) y en un tiempo razonable.

En concreto, lo que vamos a hacer es tomar un 1% del training (que serán unas 2000 instancias). De la muestra obtenida, reservaremos un 80% para el split de train (unas 1600 instancias) y el resto para validación y test. Este último con solo 10 instancias.

Vamos a liberar primero la memoria ocupada por todo el dataset:

In [3]:
del(dataset)

Ahora creamos el nuevo dataset tomando únicamente un 1% del training. De esta muestra, obtenemos también los conjuntos de validación y test.

In [4]:
# únicamente tomamos un 1% del training
dict_dataset = load_dataset("xsum", split='train[:1%]').shuffle(seed=42)

# De esa misma muestra, vamos a obtener otros dos subconjuntos
dict_dataset = dict_dataset.train_test_split(test_size=0.2, shuffle=False)

SIZE_TEST=  dict_dataset["test"].num_rows
# en el conjunto de validación, desde la instancia 10 hasta el final del test
dict_dataset["validation"] = dict_dataset["test"].select(range(10,SIZE_TEST))

# en el conjunto test, únicamente guardamos las 10 primeras
dict_dataset["test"] = dict_dataset["test"].select(range(10))

dict_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1632
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 398
    })
})

Vamos a mostrar algunas instancias

In [5]:
import random
index = random.randint(0, dict_dataset['train'].num_rows)
print('Texto original:\n\t',dict_dataset['train'][index]['document'])
print()
print('Resumen:\n\t', dict_dataset['train'][index]['summary'])


Texto original:
	 The closures from 20:00 to 06:00 BST from Monday should only affect traffic in one direction, although at times both tunnels may be closed.
Newport's A48 Southern Distributor Road will be used for diversions.
Economy and Infrastructure Secretary Ken Skates said there was an "ongoing commitment" to improving the motorway.
The work is due to be carried out mainly at night until February 2018, with the M4 scheduled to be closed between junctions 25A for Caerleon and Cwmbran and 26 at Malpas up to five nights a week.
Diversions will be put in place between junction 24 at Coldra and junction 28 at Tredegar Park for through traffic, although local traffic will be allowed to travel up to junctions 25A and 26 to access local routes.
"The M4 is of vital importance to the Welsh economy and this maintenance to the Brynglas tunnels forms part of our ongoing commitment to improving the motorway," said Mr Skates.
"The timing of this work is designed to ensure that it's carried out 

## Distribución de los tamaños de los textos

In [6]:
from transformers import AutoTokenizer

model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [7]:
len_train_texts = [len(tokenizer(text).input_ids) for text in dict_dataset['train']['document']]

import pandas as pd
df=pd.Series(len_train_texts)
df.describe(percentiles=[0.25, 0.50, 0.75, 0.85, 0.90, 0.95, 0.99])

Token indices sequence length is longer than the specified maximum sequence length for this model (738 > 512). Running this sequence through the model will result in indexing errors


count    1632.000000
mean      530.046569
std       401.863190
min        19.000000
25%       262.000000
50%       421.000000
75%       695.500000
85%       918.350000
90%      1070.900000
95%      1338.900000
99%      1903.830000
max      3013.000000
dtype: float64

El tamaño máximo en las noticias es de 3013 tokens, sin embargo vemos que el 99% de los textos tienen un tamaño menor o igual a 1903, y el 90% de los textos tienen menos de 1070 tokens.


In [8]:
len_train_sums = [len(tokenizer(text).input_ids) for text in dict_dataset['train']['summary']]

import pandas as pd
df=pd.Series(len_train_sums)
df.describe(percentiles=[0.25, 0.50, 0.75, 0.85, 0.90, 0.95, 0.99])

count    1632.000000
mean       30.498775
std         8.080602
min         3.000000
25%        25.000000
50%        30.000000
75%        35.000000
85%        38.000000
90%        40.000000
95%        44.000000
99%        54.690000
max       129.000000
dtype: float64

El tamaño máximo en los resúmenes es de 129 tokens, sin embargo vemos que el 99% de los resúmenes tienen un tamaño menor o igual a 54, y el 90% de los textos tienen menos de 40 tokens.


### Tokenization

Como mucho otros transformers, el modelo T5 necesita que todas sus entradas tengan el mismo tamaño, y el tamaño máximo que admite es 512 tokens.
Los recursos informáticos aumentan cuadráticamente con respecto a la longitud de la secuencia de entrada. Por tanto, esto aumenta el tiempo de entrenamiento y el consumo de memoria.


In [9]:
import transformers
transformers.__version__

'4.34.0'

In [10]:
MAX_INPUT_LENGTH = 512  #  Tamaño máximo para la entrada del modelo (si la versión de transformeres es 5, puedes usar hasta 1024)
MAX_TARGET_LENGTH = 129  # Tamaño máximo de la salida generada por el modelo

Volvemos a crear el objeto tokenizador, pero ahora indicando la longitud maxima.

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=MAX_INPUT_LENGTH)
print(tokenizer.model_max_length)


512


También vamos a definir la función tokenize, que es ligeramente distinta a las usadas para el modelo BERT.

Una de las características de T5 es que como BERT puede ser ajustado a distintas tareas de NLP, pero es necesario indicarle el nombre de la tarea que va a realizar. En nuestro caso, tendremos que añadir a cada entrada, el prefijo "summarize: ".


In [12]:
PREFIX='summarize: '

def tokenize(examples):

    # añadimos el prefijo a cada texto de la entrada
    inputs = [PREFIX + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, padding=True, truncation=True, return_tensors="pt").to('cuda')

    # también tenemos que tokenizar los resúmenes (que serán nuestras labels)
    labels = tokenizer(text_target=examples["summary"], max_length=MAX_TARGET_LENGTH,  padding=True, truncation=True, return_tensors="pt").to('cuda')

    # añadimos a la codificación de las entrada, model_inputs, un nuevo campo 'labels' que contiene
    # los input_ids de los tokens del resumen
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# we apply the function to the dataset for encoding it
encoded_datasets = dict_dataset.map(tokenize, batched=True)
encoded_datasets

Map:   0%|          | 0/1632 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/398 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1632
    })
    test: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 398
    })
})

Ahora vamos a borrar las columnas que no utilizará el transformer (document, summary, e id).

El módelo únicamente trabajará con los input-ids de los textos de entrada, su capa de attention_mask (distingue entre padding y token normal), y las labels (que son los input_ids de los tokens del resumen)

In [13]:
encoded_datasets=encoded_datasets.remove_columns(['document', 'summary', 'id'])
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1632
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 398
    })
})

## Modelo (Pytorch)

Vamos a utilizar la clase **AutoModelForSeq2SeqLM** que nos permite cargar un modelo y extender su arquitectura **seq2seq**. Esta arquitectura, compuesta por un codificador y un decodificador, es apropiada para las tareas de generación de textos, ya que reciben una secuencia de entrada (un texto) y deben generar una secuencia de salida (un resumen, por ejemplo).


In [14]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')


Como siempre tenemos que definir los hiper-parámetros y las métricas. Respecto a los hiper-parámetros usaremos la clase **Seq2SeqTrainingArguments** que ya nos proporciona un conjunto de hiperparámetros apropiados para esta tarea. Vamos a modificar algunos de ellos, como por ejemplo, el tamaño del batch y el número de epochs:

In [15]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir='./outputs',
    evaluation_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

Respecto a las métricas, usaremos las propias para la tarea de generación de resúmenes, como Rouge, que están implementadas en la librería **keras_nlp**. En concreto, vamos a usar rougeL, que es la métrica más común para este tipo de sistemas, y da una una puntuación basada en la longitud de la subsecuencia común más larga presente en el texto de referencia (resumen) y el texto generado.


In [16]:
import keras_nlp
rouge_L = keras_nlp.metrics.RougeL()

def compute_metrics(eval_predictions):
    #tomamos las salidas del modelo y sus respectivas labels (son input_ids)
    predictions, labels = eval_predictions

    # para poder aplicar rougeL, tenemos que pasarlas a texto, para eso usamos el método
    # batch_decode, que decodificará todas las predicciones. Con el parámetro skip_special_tokens,
    # ignoramos los tokens especiales
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # También tenemos que decodificar los input_ids de las labels, es decir,
    # si algún id es < 0, lo reemplazamos por el id del token pad.
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens

    # ahora ya podemos pasar de ids a texto, también usando la misma función e ignorando los tokens especiales
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ya tenemos las prediciones y las labels transformadas a texto, y ahora así, podemos
    # calcular la métrica con la función rouge_L proporcionada por la librería keras_nlp
    result = rouge_L(decoded_labels, decoded_predictions)
    # rouge_L devuelve precisión, recall y F1, nosotros únicamente vamos a usar su f1
    # creamos un diccionario con el nombre de la métrica, y el valor de rouge_L f1
    result = {"RougeL": result["f1_score"]}

    # devolvemos el diccionario
    return result

Using TensorFlow backend


Para pasarle los datos al modelo, necesitamos crear un data collator específico para la tarea. Para ello usaremos, la clase **DataCollatorForSeq2Seq** :

In [17]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Ahora ya sí podemos crear el trainer, que en este caso será un objeto de la clase *Seq2SeqTrainer*. Como siempre tenemos que pasarle el modelo, los argumentos, la función que calcula las métricas, los datos de conjunto training y validación codificados, el tokenizador y el data collator para transferir los datos al modelo:

In [18]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Entrenamos (tardará unos minutos):

In [19]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rougel
1,No log,3.889640,"tf.Tensor(0.11931312, shape=(), dtype=float32)"


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "0.11931312084197998" of type <class 'tensorflow.python.framework.ops.EagerTensor'> for key "eval/RougeL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=102, training_loss=7.711432363472733, metrics={'train_runtime': 73.0489, 'train_samples_per_second': 22.341, 'train_steps_per_second': 1.396, 'total_flos': 220877820002304.0, 'train_loss': 7.711432363472733, 'epoch': 1.0})

Vamos a evaluar el modelo sobre el conjunto de validación:

In [20]:
trainer.evaluate()

Trainer is attempting to log a value of "0.11931321769952774" of type <class 'tensorflow.python.framework.ops.EagerTensor'> for key "eval/RougeL" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 3.8896400928497314,
 'eval_RougeL': <tf.Tensor: shape=(), dtype=float32, numpy=0.11931322>,
 'eval_runtime': 19.3016,
 'eval_samples_per_second': 20.62,
 'eval_steps_per_second': 1.295,
 'epoch': 1.0}

El modelo obtiene una rouge_L (f1) de 0.119 sobre el conjunto de validación. Una puntuación de ROUGE cercana a cero indica poca similitud entre los resúmenes generados y los resúmenes de referencia.

Seguramente si utilizamos más datos para entrenar y también incrementamos el número de epochs, este valor aumentaría.



## Evaluación sobre el conjunto test

Nuestro conjunto test es muy pequeño, pero aun así también vamos a calcular la métrica rouge-L sobre dicho conjunto.

Para aplicar el modelo sobre dicho conjunto, vamos a encapsularlo en un pipeline.
Antes de evaluarlo, vamos a aplicarlo sobre uno de los textos en el test para ver qué resumen genera:


In [21]:
from transformers import pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="pt", device=0)

input_text = dict_dataset["test"][0]["document"]
print("Texto:", input_text)
summarizer(
    input_text,
    min_length=5,
    max_length=120,
    # max_new_tokens=MAX_TARGET_LENGTH,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors


Texto: Jose Fonte's first goal for 18 months gave the hosts the lead, glancing in a header from Dusan Tadic's corner.
Virgil van Dijk earlier saw a header cleared off the line, but he doubled the lead with a close-range prod.
Vardy headed the Foxes back into the match, before blasting home his ninth of the season in injury time to keep the Foxes in fifth.
Relive the match action here
All the Premier League action and reaction
Not judging by their second-half display.
The Foxes have scored in every Premier League match this season and, sparked into life by the half-time introduction of forwards Riyad Mahrez and Nathan Dyer, they earned an unlikely point with a stunning final 45 minutes.
Southampton were in complete control at half-time but, helped by the trickery of Mahrez and the clinical finishing of Vardy, the Foxes again showed they should never be ruled out.
The draw is the seventh point Leicester have earned from a losing position this season.
It would be very hard to leave the Le

[{'summary_text': "Virgil van Dijk's first goal for 18 months gave the hosts the lead . he doubled the lead with a header from Dusan Tadic's corner . the 28-year-old is the fourth englishman to score in six consecutive matches this season ."}]

¿Qué te parece el resumen?.

Ahora sí vamos calcular las métricas sobre el conjunto test. Para ello aplicamos el pipeline sobre todo el conjunto test. Tardará un par de minutos (y eso que son solo 10 textos!!):

In [22]:
generated_summaries =summarizer(dict_dataset["test"]["document"], truncation=True, min_length=5, max_length=MAX_TARGET_LENGTH)
generated_summaries

Your max_length is set to 129, but your input_length is only 91. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)


[{'summary_text': "Virgil van Dijk doubled the lead with a header from Dusan Tadic's corner . he scored his ninth of the season in injury time to keep the Foxes in fifth . the hosts have scored in every premier league match this season ."},
 {'summary_text': 'images of a million or more Belgians trudging along the roads to the Netherlands or France have been strikingly similar each time . in 1914, cinema audiences across Britain, many of whom had probably thought this kind of thing would never happen in Europe again, watched jerky black and white newsreel pictures .'},
 {'summary_text': 'Hurtado joined the Royals from Pacos de Ferreira last summer . the 26-year-old has made 25 appearances for Peru, scoring two goals .'},
 {'summary_text': 'the prime minister received nearly $700m (£455m) in his bank account from a generous donor or donors . one Muslim official suggested that their party needed the funds to counter the "Jewish threat" in the last general election, while another said the

In [23]:
generated_summaries=[example['summary_text'] for example in generated_summaries]


Ahora que ya tenemos los resúmenes generados, podemos compararlos con los resúmenes de referencia y calcular rouge_L:

In [24]:
result = rouge_L(dict_dataset["test"]["summary"], generated_summaries)
result

{'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.15695055>,
 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.09936103>,
 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.119162194>}

Como comentabamos antes rouge_L, además de F1, también prorporciona la precisión y recall. Los tres valoes son muy bajos lo que indica es que hay muy poca similitud entre los resúmenes generados y los resúmenes de referencia (como ocurría en el conjunto validación).

Para tomar el valor de cada tensor, puedes usar el siguiente código:

In [25]:
import tensorflow as tf
print('Precision (rouge_L):', tf.get_static_value(result['precision']))
print('recall (rouge_L):', tf.get_static_value(result['recall']))
print('f1_score (rouge_L):', tf.get_static_value(result['f1_score']))


Precision (rouge_L): 0.15695055
recall (rouge_L): 0.09936103
f1_score (rouge_L): 0.119162194


Los resultados que hemos obtenido son muy pobres. Esto se debe principalmente a que hemos utilizado un conjunto para entrenar demasiado pequeño (únicamente 1600 pares de textos). Esto lo hemos hecho para que el modelo pueda entrenarse en poco tiempo y sin necesidad de usar el servicio Google Colab Pro.
Si estás interesado en la tarea de generación de resúmenes, te recomiendo que trates de utilizar todo el dataset o bien una muestra mayor al 1% (que es el tamaño que hemos usado en este ejercicio).

Además, también te recomiendo que experimentes con el conjunto de hiperparámetros y en particular, que aumentes el número de epochs. Probablemente los resultados aumenten significativamente (aunque también lo hará el tiempo de entrenamiento).



Hay un nuevo modelo de T5, LongT5 (https://huggingface.co/docs/transformers/model_doc/longt5), que usa el mecanimso Transient Global (TGlobal), y que obtiene mejores resultados en la tarea de generación de resúmenes. Su principal ventaja es que permite manejar entradas de hasta
16.384 tokens.
